In [73]:
import datetime
import json
import requests

import pandas as pd

from requests_oauthlib import OAuth1Session
from urllib import parse

In [7]:
# Define key and token
dict_key = {}

with open('../../key-and-tokes.txt', 'r') as f:
    for line in f.readlines():
        pair = line.replace('\n', '').split(':')
        dict_key[pair[0].strip()] = pair[1].strip()

## 試しに自分のアカウントのtweetを取得

In [17]:
# Set url
url = 'https://api.twitter.com/1.1/statuses/user_timeline.json'

# Set parameters
params = {'screen_name':'gan_informatics',
          'exclude_replies' : True,
          'include_rts' : False,
          'count' : 200}

In [18]:
# API authentication
twitter = OAuth1Session(dict_key['Consumer Key'], dict_key['Consumer Secret'],
                        dict_key['Access Token'], dict_key['Access Token Secret'])

In [19]:
# Request
res = twitter.get(url, params=params)

In [25]:
# Get tweets
with open('../../data/twitter/sample_tweets.txt', 'w', encoding='UTF-8') as f:
    for j in range(100):
        res = twitter.get(url, params=params)

        if res.status_code == 200:

            # API残り
            limit = res.headers['x-rate-limit-remaining']
            print ("API remain: " + limit)
            if limit == 1:
                sleep(60*15)

            n = 0
            timeline = json.loads(res.text)
            # 各ツイートの本文を表示
            for i in range(len(timeline)):
                if i != len(timeline)-1:
                    f.write(timeline[i]['text'] + '\n')
                else:
                    f.write(timeline[i]['text'] + '\n')
                    #一番最後のツイートIDをパラメータmax_idに追加
                    params['max_id'] = timeline[i]['id']-1

API remain: 897
API remain: 896
API remain: 895
API remain: 894
API remain: 893
API remain: 892
API remain: 891
API remain: 890
API remain: 889
API remain: 888
API remain: 887
API remain: 886
API remain: 885
API remain: 884
API remain: 883
API remain: 882
API remain: 881
API remain: 880
API remain: 879
API remain: 878
API remain: 877
API remain: 876
API remain: 875
API remain: 874
API remain: 873
API remain: 872
API remain: 871
API remain: 870
API remain: 869
API remain: 868
API remain: 867
API remain: 866
API remain: 865
API remain: 864
API remain: 863
API remain: 862
API remain: 861
API remain: 860
API remain: 859
API remain: 858
API remain: 857
API remain: 856
API remain: 855
API remain: 854
API remain: 853
API remain: 852
API remain: 851
API remain: 850
API remain: 849
API remain: 848
API remain: 847
API remain: 846
API remain: 845
API remain: 844
API remain: 843
API remain: 842
API remain: 841
API remain: 840
API remain: 839
API remain: 838
API remain: 837
API remain: 836
API rema

## 特定の単語を含むtweetを検索

In [21]:
search_query = '"川中島" "幻舞"'

parse.quote(search_query)

'%22%E5%B7%9D%E4%B8%AD%E5%B3%B6%22%20%22%E5%B9%BB%E8%88%9E%22'

In [86]:
# Set url
url = 'https://api.twitter.com/1.1/search/tweets.json'

search_query = '"川中島" "幻舞"'

# Set parameters
params = {'q': parse.quote(search_query),
          'exclude_replies' : True,
          'include_rts' : False,
          'count' : 200}

max_ite = 10

txt_output = '../../data/twitter/kawanakajima_genbu.txt'

In [50]:
# API authentication
twitter = OAuth1Session(dict_key['Consumer Key'], dict_key['Consumer Secret'],
                        dict_key['Access Token'], dict_key['Access Token Secret'])

In [87]:
df_tweets = pd.DataFrame(columns=['id', 'favorite_count', 'retweet_count', 'text'])

for ite in range(max_ite):
    # Request
    res = twitter.get(url, params=params)
    
    #Confirm response
    if res.status_code == 200:
        # Output tweet
        tweets = json.loads(res.text)['statuses']
        for tweet in tweets:
            df_tweets = df_tweets.append(pd.DataFrame(
                {'id': tweet['id'], 'favorite_count': tweet['favorite_count'],
                 'retweet_count': tweet['retweet_count'], 'text': tweet['text'].replace('\n', u'。')},
            index=[0]), ignore_index=True)
        
        # Print remaining API
        remaining_api = int(res.headers['x-rate-limit-remaining'])
        print('Iteration: %s/%s, Remaining API: %s'%(ite+1, max_ite, remaining_api))
        
        # Sleep 15 minutes if there is no remain
        if remaining_api <= 1:
            print('There is no remaining API.\nSleeping until %s'%(datetime.datetime.now() + datetime.timedelta(minutes=15)))
            
# Output df_tweets
df_tweets.loc[:, ['id', 'favorite_count', 'retweet_count', 'text']].to_csv(txt_output, sep='\t', index=False, encoding='utf-8')

Iteration: 0/10, Remaining API: 77
Iteration: 1/10, Remaining API: 76
Iteration: 2/10, Remaining API: 75
Iteration: 3/10, Remaining API: 74
Iteration: 4/10, Remaining API: 73
Iteration: 5/10, Remaining API: 72
Iteration: 6/10, Remaining API: 71
Iteration: 7/10, Remaining API: 70
Iteration: 8/10, Remaining API: 69
Iteration: 9/10, Remaining API: 68


In [88]:
params

{'count': 200,
 'exclude_replies': True,
 'include_rts': False,
 'q': '%22%E5%B7%9D%E4%B8%AD%E5%B3%B6%22%20%22%E5%B9%BB%E8%88%9E%22'}

In [92]:
tweets[0]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Wed Nov 29 14:26:12 +0000 2017',
 'entities': {'hashtags': [],
  'media': [{'display_url': 'pic.twitter.com/UeqZE4v8Gr',
    'expanded_url': 'https://twitter.com/oriolus_lover/status/935874816011206658/photo/1',
    'id': 935874807966588928,
    'id_str': '935874807966588928',
    'indices': [78, 101],
    'media_url': 'http://pbs.twimg.com/media/DPzlLesV4AAofm1.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DPzlLesV4AAofm1.jpg',
    'sizes': {'large': {'h': 2048, 'resize': 'fit', 'w': 1536},
     'medium': {'h': 1200, 'resize': 'fit', 'w': 900},
     'small': {'h': 680, 'resize': 'fit', 'w': 510},
     'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
    'source_status_id': 935874816011206658,
    'source_status_id_str': '935874816011206658',
    'source_user_id': 893774388679499776,
    'source_user_id_str': '893774388679499776',
    'type': 'photo',
    'url': 'https://t.co/UeqZE4v8Gr'}],
  'symbols': [],
  'ur

In [95]:
tweets[0]['in_reply_to_user_id'] is None

True

In [93]:
tweets[0]['retweeted']

False

In [77]:
print('There is no remaining API.\nSleeping until %s'%(datetime.datetime.now() + datetime.timedelta(minutes=15)))

There is no remaining API.
Sleeping until 2017-11-30 17:05:49.424602


In [43]:
# Set text file for output
pd.DataFrame(columns=['id', 'favorite_count', 'retweet_count', 'text']).to_csv(sep='\t', index=False)

# Get tweets
with open('../../data/twitter/kawanakajima_genbu.txt', 'w', encoding='UTF-8') as f:
    for j in range(100):
        # Request
        res = twitter.get(url, params=params)

        if res.status_code == 200:

            # Remaining API
            limit = res.headers['x-rate-limit-remaining']
            print('API remain: %s'%(limit))
            if limit == 1:
                # Sleep 15 minutes
                sleep(60*15)

            n = 0
            timeline = json.loads(res.text)['statuses']
            
            # Write text
            for i in range(len(timeline)):
                f.write(timeline[i]['text'] + '\n')
                    
            # Add max_id as a parameter
            params['max_id'] = json.loads(res.text)['search_metadata']['max_id']

API remain: 178
API remain: 177


IndexError: list index out of range

In [63]:
res.status_code

200

In [51]:
res = twitter.get(url, params=params)

timeline = json.loads(res.text)['statuses']

In [60]:
timeline[0]['text'].replace('\n', u'。')

'RT @oriolus_lover: 5杯目。川中島 幻舞。しっかりしたボディ感。それでありながらスッキリして飲みやすいです(^_^)。これも美味しい‼️ https://t.co/UeqZE4v8Gr'

In [56]:
timeline[0]['favorite_count']

0

In [57]:
timeline[0]['retweet_count']

1

In [58]:
timeline[0]['id']

935877557429211136